In [ ]:
from esperanto import AIFactory

# Create a LangChain-compatible model
model = AIFactory.create_language("openai", "gpt-4o-mini").to_langchain()

## Test Single Question Generation

In [ ]:
from quiz_me import generate_question, SingleQuestionConfig, QuestionType

# Sample content to generate questions from
content = """
Python is a high-level, interpreted programming language known for its clear syntax 
and readability. It was created by Guido van Rossum and first released in 1991. 
Python supports multiple programming paradigms, including procedural, object-oriented, 
and functional programming. It has a large standard library and an active community 
that contributes thousands of third-party packages through PyPI (Python Package Index).
Python is widely used in web development, data science, machine learning, automation, 
and scientific computing.
"""

# Generate a single multiple choice question
config = SingleQuestionConfig(
    content=content,
    question_type=QuestionType.MULTIPLE_CHOICE,
    generator_model=model,
)

result = await generate_question(config)
print(f"Question: {result.question.statement}")
print(f"Type: {result.question.question_type.value}")
print(f"Alternatives: {result.question.alternatives}")
print(f"Correct Answer: {result.question.correct_answer}")
print(f"Explanation: {result.question.explanation}")
print(f"Generation time: {result.generation_time_ms}ms")

In [ ]:
# Generate an open-ended question
config_open = SingleQuestionConfig(
    content=content,
    question_type=QuestionType.OPEN_ENDED,
    generator_model=model,
)

result_open = await generate_question(config_open)
print(f"Question: {result_open.question.statement}")
print(f"Type: {result_open.question.question_type.value}")
print(f"\nGrading Rubric:\n{result_open.question.correct_answer}")
print(f"\nExplanation/Model Answer:\n{result_open.question.explanation}")

## Test Multi-Question Generation

In [ ]:
from quiz_me import generate_questions, MultiQuestionConfig, QuestionTypeMix

# Generate multiple questions with a specific mix
config_multi = MultiQuestionConfig(
    content=content,
    num_questions=3,
    question_mix=[
        QuestionTypeMix(question_type=QuestionType.MULTIPLE_CHOICE, count=2),
        QuestionTypeMix(question_type=QuestionType.OPEN_ENDED, count=1),
    ],
    generator_model=model,
)

result_multi = await generate_questions(config_multi)

print(f"Generated {len(result_multi.questions)} questions in {result_multi.generation_time_ms}ms\n")
print("Plan:")
for i, spec in enumerate(result_multi.plan.question_specs):
    print(f"  {i+1}. {spec.question_type.value} - {spec.focus_area}")

print("\n" + "="*60 + "\n")

for i, q in enumerate(result_multi.questions):
    print(f"Question {i+1} ({q.question_type.value}):")
    print(f"  {q.statement}")
    if q.alternatives:
        print(f"  Alternatives: {q.alternatives}")
        print(f"  Correct: {q.correct_answer}")
    print()

## Test with Supervision

Use a supervisor model to review generated questions.

In [ ]:
# Create a supervisor model (can be the same or different from generator)
supervisor_model = AIFactory.create_language("openai", "gpt-4o-mini").to_langchain()

# Generate with supervision enabled
config_supervised = SingleQuestionConfig(
    content=content,
    question_type=QuestionType.MULTIPLE_CHOICE,
    generator_model=model,
    supervisor_model=supervisor_model,
    supervision_enabled=True,
    generator_instructions="Create challenging questions that test deep understanding.",
    supervisor_instructions="Ensure the question is unambiguous and all alternatives are plausible.",
)

result_supervised = await generate_question(config_supervised)
print(f"Question: {result_supervised.question.statement}")
print(f"Approved: {result_supervised.question.approved}")
print(f"Retries used: {result_supervised.retries_used}")
if result_supervised.question.supervision_feedback:
    print(f"Feedback: {result_supervised.question.supervision_feedback}")

## Topic-Based Generation

Generate questions from a topic using the model's knowledge (no content provided).

In [ ]:
# Generate from a topic (no content, uses model knowledge)
config_topic = SingleQuestionConfig(
    topic="The French Revolution and its impact on European politics",
    question_type=QuestionType.MULTIPLE_CHOICE,
    generator_model=model,
)

result_topic = await generate_question(config_topic)
print(f"Question: {result_topic.question.statement}")
print(f"Alternatives: {result_topic.question.alternatives}")
print(f"Correct: {result_topic.question.correct_answer}")
print(f"Explanation: {result_topic.question.explanation}")

## Multi-Language Support

Generate questions in different languages.

In [ ]:
# Generate a question in Portuguese
portuguese_content = """
Python é uma linguagem de programação de alto nível, interpretada e de propósito geral.
Foi criada por Guido van Rossum e lançada pela primeira vez em 1991.
Python é conhecida por sua sintaxe clara e legível, sendo amplamente utilizada em
ciência de dados, inteligência artificial, desenvolvimento web e automação.
"""

config_pt = SingleQuestionConfig(
    content=portuguese_content,
    question_type=QuestionType.MULTIPLE_CHOICE,
    generator_model=model,
    language="Portuguese",  # Generate all content in Portuguese
)

result_pt = await generate_question(config_pt)
print(f"Pergunta: {result_pt.question.statement}")
print(f"Alternativas: {result_pt.question.alternatives}")
print(f"Resposta Correta: {result_pt.question.correct_answer}")
print(f"Explicação: {result_pt.question.explanation}")

## Question Improvement

Improve an existing question based on feedback.

In [ ]:
from quiz_me import improve_question

# Use the first question we generated as the original
original_question = result.question

print("Original Question:")
print(f"  Statement: {original_question.statement}")
print(f"  Alternatives: {original_question.alternatives}")
print()

# Improve based on feedback
config_improve = SingleQuestionConfig(
    content=content,
    question_type=original_question.question_type,
    generator_model=model,
)

improved_result = await improve_question(
    question=original_question,
    feedback="Make the distractors more challenging by including plausible alternatives that test deeper understanding.",
    config=config_improve,
)

print("Improved Question:")
print(f"  Statement: {improved_result.question.statement}")
print(f"  Alternatives: {improved_result.question.alternatives}")
print(f"  Correct Answer: {improved_result.question.correct_answer}")
print(f"  Generation time: {improved_result.generation_time_ms}ms")